In [1]:
import requests
import apache_beam as beam
from beam_postgres.io import WriteToPostgres

class CallAPI(beam.DoFn):
    def process(self, element):
        url = 'http://127.0.0.1:8000/content/'
        res = requests.get(url)
        data_list = res.json()
        
        for data in data_list:
            yield {
                'datetime': data['datetime'],
                'street_time': data['street_time'],
                'count': data['count'],
                'velocity': data['velocity']
            }

# Create a pipeline
with beam.Pipeline() as p:
    data = p | beam.Create(values=[{'from': 1, 'to': 100}])
    ap_response = data | beam.ParDo(CallAPI())
    #ap_response | "Print results" >> beam.Map(print)
    
    

    # Write the ap_response data to PostgreSQL
    ap_response | "Write to PostgreSQL" >> WriteToPostgres(
        host='localhost',
        database='transport',
        user='postgres',
        password='postgres',
        table='your_table_name',
        batch_size=1000,  # Adjust batch size as needed
    )

    #Run the pipeline
    result = p.run()


INFO:beam_postgres.client:Successfully execute query: INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-31 21:15:00', 1266.0, 21, 82.0952380952381);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-09 07:15:00', 1222.0, 23, 84.6086956521739);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-04 06:00:00', 145.0, 2, 14.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-12 12:30:00', 2188.0, 10, 33.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 03:15:00', 470.0, 2, 9.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 06:15:00', 2818.0, 65, 69.61538461538461);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 07:45:00', 10.0, 1, 23.0);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-14 16:15:00', 